# Experiments - Data Preparation

In [1]:
%cd /Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/

/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions


In [2]:
import sys
import os
src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(src_dir)

from read_data.read_data import read_urls_questions, get_questions, get_url_content_tuples, get_relevant_docs
from elasticsearch import Elasticsearch
from retrieval.sparse_retrieval.bm25 import set_index, get_result_tuples

### Naive method

### Retrieval methods

In [3]:
collected, questions =  read_urls_questions(os.path.join('data/amsterdam/amsterdam_full.csv'),
                                            os.path.join('data/question_answer/questions.csv')) # read collected urls and questions + remove unsuccessful collection

In [4]:
len(collected)

8908

In [5]:
len(questions)

19134

#### Make passages

In [6]:
import nltk
import pandas as pd

# Download the NLTK tokenizer data (if not already downloaded)
nltk.download('punkt')

# Create an empty dataframe for the new data
new_df = pd.DataFrame(columns=['URL', 'Textual_Content'])

# Iterate over each row in the original dataframe
for index, row in collected.iterrows():
    url = row['URL']
    content = str(row['Textual_Content'])
    
    # Tokenize the content into sentences
    sentences = nltk.sent_tokenize(content, language='dutch')
    
    # Create passages by combining sentences until the total word count reaches 100
    passages = []
    passage_words = []
    for sentence in sentences:
        sentence_words = nltk.word_tokenize(sentence, language='dutch')
        if len(passage_words) + len(sentence_words) <= 100:
            passage_words.extend(sentence_words)
        else:
            passages.append(' '.join(passage_words))
            passage_words = sentence_words
    
    # Add the last remaining passage (if any)
    if passage_words:
        passages.append(' '.join(passage_words))
    
    # Create a new dataframe for the passages
    passage_df = pd.DataFrame({'URL': url, 'Textual_Content': passages})
    
    # Concatenate the new dataframe with the main dataframe
    new_df = pd.concat([new_df, passage_df], ignore_index=True)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/natalipeeva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
len(new_df)

65103

In [14]:
new_df.head()

,URL,Textual_Content
0,https://www.amsterdam.nl/veelgevraagd/,Veelgevraagd - Gemeente Amsterdam Direct naar ...
1,https://www.amsterdam.nl/veelgevraagd/,Regel het online Veelgevraagd Melding openbare...
2,https://www.amsterdam.nl/veelgevraagd/,Contactformulier Bel het telefoonnummer 14 020...
3,https://www.amsterdam.nl/veelgevraagd/?categor...,Thema - Burgerzaken - Gemeente Amsterdam Direc...
4,https://www.amsterdam.nl/veelgevraagd/?categor...,Verhuizen binnen Amsterdam Ik ben uitgeschreve...


#### Random

In [16]:
from retrieval.random_retrieval.random_retrieve import perform_random_search

In [18]:
question_list = get_questions(questions)
document_list = get_url_content_tuples(new_df)

In [19]:
all_results_documents_random = perform_random_search(question_list, document_list, k=10) # took 7 mins 

#### BM25

In [20]:
es_client = Elasticsearch("http://localhost:9200")

In [21]:
es_client.info()

ObjectApiResponse({'name': '765e69f517ee', 'cluster_name': 'docker-cluster', 'cluster_uuid': 't1JOUdrjSOanv_1Dp5j5pg', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [22]:
question_list = get_questions(questions)

mappings = {
        "properties": {
            "url": {"type": "text"},
            "text": {"type": "text", "analyzer": "standard", "similarity": "BM25"}
    }
}

In [23]:
es_client = set_index(es_client=es_client, collected=new_df, mappings=mappings) # 1 min

In [24]:
results_bm25 = get_result_tuples(es_client=es_client, questions=question_list, n=10) # 15 gives the best results ; when i set it to 5 gives 0,26

#### Result Evaluation

In [28]:
questions2 = pd.read_csv('data/amsterdam/only_amsterdam_questions.csv')

In [29]:
questions2.head()

,Unnamed: 0.1,Unnamed: 0,Year,Month,Question,Answer,Document,URLs,Cleaned_URLs
0,1,1,2020,6,\n7.\nKan het college de reeds bestaande zwemp...,\nVoor het vinden van de officiële zwemplekken...,https://amsterdam.raadsinformatie.nl/document/...,['https://www.amsterdam.nl/veelgevraagd/?casei...,['https://www.amsterdam.nl/veelgevraagd/?casei...
1,6,7,2021,8,\n \n3. Huisartsen geven aan meer informatie n...,"\nDe uitvoerder van de regeling, het CAK, lij...",https://amsterdam.raadsinformatie.nl/document/...,['https://www.amsterdam.nl/zorg-ondersteuning/...,['https://www.amsterdam.nl/zorg-ondersteuning/...
2,7,8,2021,8,\n \n8. Weten ongedocumenteerden de weg naar m...,"\nDe Kruispost wordt goed bezocht, maar het c...",https://amsterdam.raadsinformatie.nl/document/...,['https://www.amsterdam.nl/zorg-ondersteuning/...,['https://www.amsterdam.nl/zorg-ondersteuning/...
3,9,14,2022,7,\n \n4. Is het college tot nu toe tevreden met...,"\nJa, met de beschikbare middelen is de uitvo...",https://amsterdam.raadsinformatie.nl/document/...,['https://www.amsterdam.nl/wonen-leefomgeving/...,['https://www.amsterdam.nl/wonen-leefomgeving/...
4,10,16,2019,10,\n \n1. \nKan aan de werkinstructie worden toe...,...,https://amsterdam.raadsinformatie.nl/document/...,['https://www.amsterdam.nl/privacy/loket/'],['https://www.amsterdam.nl/privacy/loket/']


In [53]:
list(questions2['Question'])[1]

'\n \n3. Huisartsen geven aan meer informatie nodig te hebben over declaratiemogelijkheden van de \ngeboden zorg. Is het college bereid om te onderzoeken hoe de gemeente hierin beter kan \nvoorzien?  \n \n'

In [51]:
list(questions2['Cleaned_URLs'])[1]

"['https://www.amsterdam.nl/zorg-ondersteuning/ondersteuning/vluchtelingen/ongedocumenteerden/']"

In [75]:
list(results_bm25.keys())[9999]

"\n \n4. Wat is de rol en wat zijn de verantwoordelijkheden van de gemeente bij het bepalen van \neen CTER-code of soortgelijke code en de plaatsing van zo'n code bij een persoon? \n \n"

In [76]:
results_bm25[list(results_bm25.keys())[9999]][1]

[{'url': 'https://www.amsterdam.nl/bestuur-organisatie/gemeenteraad/werkt-raad/het-presidium-van-de/',
  'text': 'Het presidium van de gemeenteraad - Gemeente Amsterdam Direct naar inhoud GemeenteAmsterdam AAA Mijn Amsterdam English site Menuzoeken Onderwerpen Nieuws Contact Zoeken in Amsterdam.nl Zoek Zoek Verbergen Deze browser wordt niet meer ondersteund . Gebruik een recente versie van Edge , Chrome of Firefox . Pad tot huidige pagina Home Bestuur en Organisatie Gemeenteraad Hoe werkt de raad ? Het presidium van de gemeenteraad Het dagelijks bestuur van de gemeenteraad is het presidium . Het presidium houdt zich voornamelijk bezig met zaken van bedrijfsvoeringstechnische en huishoudelijke aard . Het bereidt bijvoorbeeld de conceptagenda van de gemeenteraad voor .',
  'passage_id': '42507',
  'score': 45.816044,
  'passage_text': 'Het presidium van de gemeenteraad - Gemeente Amsterdam Direct naar inhoud GemeenteAmsterdam AAA Mijn Amsterdam English site Menuzoeken Onderwerpen Nieuws 

In [52]:
results_bm25[list(questions2['Question'])[1]][1]

[{'url': 'https://www.amsterdam.nl/bestuur-organisatie/college/nieuws/nieuws-26-oktober-2022/',
  'text': 'Het college vindt het van belang om dit jaar nog extra aandacht te geven aan ondernemers die na de coronacrisis nog onvoldoende inkomen hebben om in hun levensonderhoud te voorzien . Deze ondernemers hebben opnieuw te maken met een onzekere periode door onder meer stijgende energieprijzen en hoge inflatie . Het college wil perspectief bieden door tijdelijk coulanter om te gaan met de afhandeling van de aanvraag voor een Bbz-uitkering . Verkeer , Vervoer en Luchtkwaliteit Het college van B en W stemt in met de raadscommissievoordracht en raadsbrief over Mokumflex .',
  'passage_id': '43021',
  'score': 27.942621,
  'passage_text': 'Het college vindt het van belang om dit jaar nog extra aandacht te geven aan ondernemers die na de coronacrisis nog onvoldoende inkomen hebben om in hun levensonderhoud te voorzien . Deze ondernemers hebben opnieuw te maken met een onzekere periode door 

In [30]:
from irmetrics.topk import recall
predictions = []
for question in list(questions2['Question']):
    if question in results_bm25.keys():
        urls = []
        for result in results_bm25[question][1]:
            urls.append(result['url'])
        predictions.append(list(set(urls)))

relevant_docs = get_relevant_docs(questions2)

true = []
for question in relevant_docs.keys():
    true.append(list(set(relevant_docs[question])))

In [31]:
import numpy as np
from irmetrics.topk import rr

# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = rr(true_values, predictions[i], k =10)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_mrr = np.mean(mrr_values)

print("Average Mean Reciprocal Rank:", average_mrr)


Average Mean Reciprocal Rank: 0.0


In [32]:
from irmetrics.topk import recall

# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = recall(true_values, predictions[i], k=10)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_recall = np.mean(mrr_values)

print("Average Recalll Rank:", average_recall)

Average Recalll Rank: 0.0


##### Top 1

##### Top 10

#### TF-IDF

In [77]:
from retrieval.sparse_retrieval.tfidf import perform_tfidf_search

In [151]:
import ast

def get_relevant_docs(df):
    """
    Convert a DataFrame with questions and URLs into a dictionary.
    Input:
        df: a pandas DataFrame with columns 'question' and 'urls'
    Output: a dictionary where the key is the question and the value is a list of URLs
    """
    question_urls_dict = {}

    for _, row in df.iterrows():
        question = row['Question']
        urls = row['Cleaned_URLs']

        if isinstance(urls, str):
            # Convert the string to a list
            urls = ast.literal_eval(urls)

        question_urls_dict[question] = urls

    return question_urls_dict


In [155]:
question_list = get_questions(questions2)
document_list = get_url_content_tuples(new_df)
all_results = perform_tfidf_search(question_list, document_list, k=100) # took 3 minutes!!!!!!

In [141]:
list(all_results.keys())[31]

'\n  \n2. Waarom wordt een GPK, inclusief medische keuring in Amsterdam niet \nkosteloos verstrekt? Waarom zijn deze kosten voor elke aanvrager gelijk, \nzonder dat wordt gekeken naar bijvoorbeeld financiële draagkracht? Graag \neen toelichting. \n \n'

In [146]:
list(questions2['Cleaned_URLs'])[35]

"['https://www.amsterdam.nl/bestuur-organisatie/organisatie/overige/acvz/?vkurl=acvz']"

In [156]:
all_results[list(all_results.keys())[35]][0]

(0.3170681702863419,
 'https://www.amsterdam.nl/bestuur-organisatie/organisatie/overige/adviesraden/commissie-persoonsgegevens-amsterdam/adviezen-cpa-2020/advies-top-400-24-september-2020/')

In [150]:
relevant_docs[list(relevant_docs.keys())[0]]

"['https://www.amsterdam.nl/veelgevraagd/?caseid=%7BD6E280FB-4A76-40A0-9B88-12B87E446FA6%7D', 'https://www.ggd.amsterdam.nl/gezond-wonen/zwemmen-open-water/']"

In [166]:
relevant_docs = get_relevant_docs(questions2)

predictions = []
for question in all_results.keys():
    urls = []
    for result in all_results[question]:
        urls.append(result[1])
    predictions.append(list((urls)))


true = []
for question in relevant_docs.keys():
    true.append(list((relevant_docs[question])))


# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = rr(true_values, predictions[i], k=10)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_mrr = np.mean(mrr_values)

print("Average Mean Reciprocal Rank:", average_mrr)


 # Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = recall(true_values, predictions[i], k=10)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_recall = np.mean(mrr_values)

print("Average Recall:", average_recall)

Average Mean Reciprocal Rank: 0.07229832572298324
Average Recall: 0.22602739726027396


/Users/natalipeeva/opt/anaconda3/lib/python3.8/site-packages/irmetrics/io.py:40: RuntimeWarning: Repeated predictions detected. This is an error unless the predictions are padded. Use np.nan as a padding token to suppress the warning for integer labels.
  warnings.warn(message, RuntimeWarning)


## TF-IDF with passages content

In [167]:
from sklearn.feature_extraction.text import TfidfVectorizer


import heapq

In [174]:
def tfidf_search(query, vectorizer, matrix, collection, k):
    """
    Perform a search over all documents with the given query using tf-idf.
    Input:
        query - an (unprocessed) query
        vectorizer: a fitted TfidfVectorizer
        matrix: the document-term matrix obtained from the fitted vectorizer
        collection: a list of tuples (document_id, document_content)
        k: the number of top search results to retrieve
    Output: a list of (document_id, score, document_content), sorted in descending relevance to the given query
    """
    # Preprocess the query
    preprocessed_query = str(query).lower().replace('\n', '')

    # Transform the query using the fitted vectorizer
    query_vector = vectorizer.transform([preprocessed_query])

    # Calculate the cosine similarity between the query and document vectors
    cosine_similarities = matrix.dot(query_vector.T).toarray().flatten()

    # Create a heap to maintain the top-k results
    results_heap = []

    # Iterate over the collection and update the heap with the top-k results
    for i, doc in enumerate(collection):
        doc_id = doc[0]
        score = cosine_similarities[i]
        document_content = doc[1]
        heapq.heappush(results_heap, (score, doc_id, document_content))
        if len(results_heap) > k:
            heapq.heappop(results_heap)

    # Sort the results in descending order of relevance (score)
    results_heap.sort(reverse=True)

    return results_heap[:k]  # Return only the top-k results


def perform_tfidf_search(queries, collection, k):
    """
    Perform TF-IDF search for each query in a list of queries.
    Input:
        queries - a list of queries
        collection: a list of tuples (document_id, document_content)
        k: the number of top search results to retrieve
    Output: a dictionary where the key is the query and the value is a list of (document_id, score, document_content) tuples
    """
    # Extract document contents from the collection
    document_contents = [str(doc[1]).lower().replace('\n', '') for doc in collection]

    # Initialize and fit the TfidfVectorizer
    vectorizer = TfidfVectorizer()
    matrix = vectorizer.fit_transform(document_contents)

    search_results = {}

    for query in queries:
        results = tfidf_search(query, vectorizer, matrix, collection, k=k)
        search_results[query] = [(result[1], result[0], result[2]) for result in results]  # Store document_id, score, and document_content

    return search_results

In [175]:
all_results_passages = perform_tfidf_search(question_list, document_list, k=100) # took 12secs minutes!!!!!!

In [177]:
all_results_passages_allquestions = perform_tfidf_search(list(questions['Question']), document_list, k=100) # 26 min

In [178]:
import pickle

In [179]:
with open ('data/thesis_draft/tfidf_results_all.pickle', 'wb') as f:
    pickle.dump(all_results_passages_allquestions, f)